In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler, Normalizer
from sklearn.pipeline import Pipeline

Using TensorFlow backend.


In [2]:
df = pd.read_csv('../train.csv')
print(df.shape)
df.head()

(10886, 12)


,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1


In [3]:
df_train = df.copy()
df_train['month'] = pd.DatetimeIndex(df_train['datetime']).month
# df_train = df[df['datetime'] < '2011-02'].copy()
# df_train = df[(df['datetime'] > month_ini) & (df['datetime'] < month_end)].copy()
df_train['day'] = pd.DatetimeIndex(df_train['datetime']).day
df_train['dayofweek'] = pd.DatetimeIndex(df_train['datetime']).dayofweek
df_train['hour'] = pd.DatetimeIndex(df_train['datetime']).hour
df_train = df_train.drop('datetime', axis=1)
df_train['casual_rush'] = df_train[['hour', 'workingday']].apply(
    lambda x: (0, 1)[
        (x['workingday'] == 1 and  11 <= x['hour'] <= 19)
        or (x['workingday'] == 0 and  10 <= x['hour'] <= 18)], axis = 1)
df_train['registered_rush'] = df_train[['hour', 'workingday']].apply(
    lambda x: (0, 1)[
        (x['workingday'] == 1 and  (6 <= x['hour'] <= 9 or 16 <= x['hour'] <= 21))
        or (x['workingday'] == 0 and  9 <= x['hour'] <= 22)], axis = 1)
# y = df_train['count'].copy()
y1 = df_train['casual'].copy()
y2 = df_train['registered'].copy()
df_train = df_train.drop(['casual','registered','count'], axis=1)
# df = df.drop(['casual','registered'], axis=1)
print(df_train.shape)
df_train.head()

(10886, 14)


,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,month,day,dayofweek,hour,casual_rush,registered_rush
0,1,0,0,1,9.84,14.395,81,0.0,1,1,5,0,0,0
1,1,0,0,1,9.02,13.635,80,0.0,1,1,5,1,0,0
2,1,0,0,1,9.02,13.635,80,0.0,1,1,5,2,0,0
3,1,0,0,1,9.84,14.395,75,0.0,1,1,5,3,0,0
4,1,0,0,1,9.84,14.395,75,0.0,1,1,5,4,0,0


In [21]:
X = df_train.values
Y = y.values
def new_model():
    n_features = 12
    # create model
    model = Sequential()
    model.add(Dense(100, input_dim=n_features, init='normal', activation='relu'))
    model.add(Dense(100, init='normal', activation='relu'))
    model.add(Dense(100, init='normal', activation='relu'))
    model.add(Dense(100, init='normal', activation='relu'))
    model.add(Dense(100, init='normal', activation='relu'))
    model.add(Dense(100, init='normal', activation='relu'))
    model.add(Dense(100, init='normal', activation='relu'))
    model.add(Dense(100, init='normal', activation='relu'))
    model.add(Dense(100, init='normal', activation='relu'))
    model.add(Dense(100, init='normal', activation='relu'))
    model.add(Dense(1, init='normal'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model
estimators = []
estimators.append(('scale', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=new_model, nb_epoch=100, batch_size=5, verbose=2)))
pipeline = Pipeline(estimators)
%time pipeline.fit(X, Y)
# kfold = KFold(n_splits=7)
# %time results = cross_val_score(pipeline, X, Y, cv=kfold, verbose=2)
# print("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Epoch 1/100
9s - loss: 19907.5540
Epoch 2/100
8s - loss: 11838.9096
Epoch 3/100
8s - loss: 8685.1277
Epoch 4/100
8s - loss: 7544.4556
Epoch 5/100
8s - loss: 6843.3583
Epoch 6/100
8s - loss: 6366.5176
Epoch 7/100
8s - loss: 6138.8524
Epoch 8/100
8s - loss: 5699.0748
Epoch 9/100
8s - loss: 5452.5231
Epoch 10/100
8s - loss: 5410.2701
Epoch 11/100
8s - loss: 4977.0655
Epoch 12/100
8s - loss: 4935.4295
Epoch 13/100
8s - loss: 4667.3148
Epoch 14/100
8s - loss: 4558.2761
Epoch 15/100
8s - loss: 4382.8802
Epoch 16/100
8s - loss: 4256.7303
Epoch 17/100
8s - loss: 4029.3953
Epoch 18/100
8s - loss: 4070.0821
Epoch 19/100
8s - loss: 3978.7025
Epoch 20/100
8s - loss: 3743.3123
Epoch 21/100
8s - loss: 3712.4375
Epoch 22/100
8s - loss: 3571.7386
Epoch 23/100
8s - loss: 3481.0406
Epoch 24/100
8s - loss: 3468.6636
Epoch 25/100
8s - loss: 3403.2343
Epoch 26/100
8s - loss: 3290.5648
Epoch 27/100
8s - loss: 3118.6463
Epoch 28/100
8s - loss: 3141.5831
Epoch 29/100
8s - loss: 3029.4809
Epoch 30/100
8s - los

Pipeline(steps=[('scale', StandardScaler(copy=True, with_mean=True, with_std=True)), ('mlp', <keras.wrappers.scikit_learn.KerasRegressor object at 0x7f4b114b1d30>)])

In [5]:
X = df_train.values
Y = y1.values
def new_model():
    n_features = 14
    # create model
    model = Sequential()
    model.add(Dense(100, input_dim=n_features, init='normal', activation='relu'))
    model.add(Dense(100, init='normal', activation='relu'))
    model.add(Dense(100, init='normal', activation='relu'))
    model.add(Dense(100, init='normal', activation='relu'))
    model.add(Dense(100, init='normal', activation='relu'))
    model.add(Dense(100, init='normal', activation='relu'))
    model.add(Dense(100, init='normal', activation='relu'))
    model.add(Dense(100, init='normal', activation='relu'))
    model.add(Dense(1, init='normal'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model
estimators = []
estimators.append(('scale', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=new_model, nb_epoch=100, batch_size=5, verbose=2)))
pipeline1 = Pipeline(estimators)
%time pipeline1.fit(X, Y)

Y = y2.values
pipeline2 = Pipeline(estimators)
%time pipeline2.fit(X, Y)

Epoch 1/100
8s - loss: 833.1294
Epoch 2/100
7s - loss: 596.2616
Epoch 3/100
8s - loss: 518.0979
Epoch 4/100
7s - loss: 468.8003
Epoch 5/100
8s - loss: 421.2218
Epoch 6/100
8s - loss: 365.9596
Epoch 7/100
8s - loss: 370.9051
Epoch 8/100
8s - loss: 325.5785
Epoch 9/100
7s - loss: 310.5682
Epoch 10/100
8s - loss: 300.5413
Epoch 11/100
7s - loss: 288.6473
Epoch 12/100
7s - loss: 291.6782
Epoch 13/100
7s - loss: 266.8039
Epoch 14/100
7s - loss: 283.6825
Epoch 15/100
8s - loss: 250.7884
Epoch 16/100
7s - loss: 258.1072
Epoch 17/100
8s - loss: 240.3190
Epoch 18/100
7s - loss: 243.0135
Epoch 19/100
7s - loss: 229.7389
Epoch 20/100
7s - loss: 223.1952
Epoch 21/100
7s - loss: 220.6618
Epoch 22/100
7s - loss: 215.2128
Epoch 23/100
7s - loss: 212.6657
Epoch 24/100
7s - loss: 206.1094
Epoch 25/100
7s - loss: 203.1507
Epoch 26/100
7s - loss: 203.1973
Epoch 27/100
7s - loss: 191.4241
Epoch 28/100
7s - loss: 191.6521
Epoch 29/100
7s - loss: 189.2100
Epoch 30/100
7s - loss: 181.0007
Epoch 31/100
8s - l

Pipeline(steps=[('scale', StandardScaler(copy=True, with_mean=True, with_std=True)), ('mlp', <keras.wrappers.scikit_learn.KerasRegressor object at 0x7fd7741f3e10>)])

## Prediction

In [6]:
df_test = pd.read_csv('../test.csv')
df_test.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed
0,2011-01-20 00:00:00,1,0,1,1,10.66,11.365,56,26.0027
1,2011-01-20 01:00:00,1,0,1,1,10.66,13.635,56,0.0000
2,2011-01-20 02:00:00,1,0,1,1,10.66,13.635,56,0.0000
3,2011-01-20 03:00:00,1,0,1,1,10.66,12.880,56,11.0014
4,2011-01-20 04:00:00,1,0,1,1,10.66,12.880,56,11.0014


In [7]:
dft = df_test.copy()
dft['month'] = pd.DatetimeIndex(dft['datetime']).month
# dft = df_test[df_test['datetime'] < '2011-02'].copy()
# dft = df_test[(df_test['datetime'] > month_ini) & (df_test['datetime'] < month_end)].copy()
dft['day'] = pd.DatetimeIndex(dft['datetime']).day
dft['dayofweek'] = pd.DatetimeIndex(dft['datetime']).dayofweek
dft['hour'] = pd.DatetimeIndex(dft['datetime']).hour
dft['casual_rush'] = dft[['hour', 'workingday']].apply(
    lambda x: (0, 1)[
        (x['workingday'] == 1 and  11 <= x['hour'] <= 19)
        or (x['workingday'] == 0 and  10 <= x['hour'] <= 18)], axis = 1)
dft['registered_rush'] = dft[['hour', 'workingday']].apply(
    lambda x: (0, 1)[
        (x['workingday'] == 1 and  (6 <= x['hour'] <= 9 or 16 <= x['hour'] <= 21))
        or (x['workingday'] == 0 and  9 <= x['hour'] <= 22)], axis = 1)
dft = dft.drop('datetime', axis=1)
print(dft.shape)
dft.head()

(6493, 14)


,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,month,day,dayofweek,hour,casual_rush,registered_rush
0,1,0,1,1,10.66,11.365,56,26.0027,1,20,3,0,0,0
1,1,0,1,1,10.66,13.635,56,0.0000,1,20,3,1,0,0
2,1,0,1,1,10.66,13.635,56,0.0000,1,20,3,2,0,0
3,1,0,1,1,10.66,12.880,56,11.0014,1,20,3,3,0,0
4,1,0,1,1,10.66,12.880,56,11.0014,1,20,3,4,0,0


In [8]:
test_pred1 = pipeline1.predict(dft)
test_pred1[test_pred1 < 0] = 0
test_pred2 = pipeline2.predict(dft)
test_pred2[test_pred2 < 0] = 0
test_pred = test_pred1 + test_pred2
str_prediction = "datetime,count\n"
for i in range(0, len(test_pred)):
    datetime = df_test['datetime'][i]
    pred = int(round(test_pred[i]))
    str_prediction += "{},{}\n".format(datetime, pred)

# print(str_prediction)
f = open('pred_keras2.csv','w')
f.write(str_prediction)
f.close()

# Month by month

In [29]:
def new_model():
    n_features = 11
    # create model
    model = Sequential()
    model.add(Dense(80, input_dim=n_features, init='normal', activation='relu'))
#     model.add(Dense(4, init='normal', activation='relu'))
    model.add(Dense(1, init='normal'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [37]:
ranges = pd.date_range('2010-12-31 23:59:59', periods=25, freq='M').tolist()
ini = True
df_train = df.copy()
dft = df_test.copy()
df_train['datetime'] = pd.to_datetime(df_train['datetime'])
dft['datetime'] = pd.to_datetime(dft['datetime'])
for i in range(0, len(ranges)-1):
    print(ranges[i+1])
    df_month = df_train[(df_train['datetime'] > ranges[i]) & (df_train['datetime'] < ranges[i+1])].copy()
    df_month['day'] = pd.DatetimeIndex(df_month['datetime']).day
    df_month['dayofweek'] = pd.DatetimeIndex(df_month['datetime']).dayofweek
    df_month['hour'] = pd.DatetimeIndex(df_month['datetime']).hour
    Y = df_month['count'].values
    df_month = df_month.drop(['casual','registered','count','datetime'], axis=1)
    X = df_month.values
    estimators = []
    estimators.append(('scale', StandardScaler()))
    estimators.append(('mlp', KerasRegressor(build_fn=new_model, nb_epoch=100, batch_size=5, verbose=0)))
    pipeline = Pipeline(estimators)
    pipeline.fit(X, Y)
    dftm = dft[(dft['datetime'] > ranges[i]) & (dft['datetime'] < ranges[i+1])].copy()
    dftm['day'] = pd.DatetimeIndex(dftm['datetime']).day
    dftm['dayofweek'] = pd.DatetimeIndex(dftm['datetime']).dayofweek
    dftm['hour'] = pd.DatetimeIndex(dftm['datetime']).hour
    dates = dftm['datetime'].tolist()
    dftm = dftm.drop('datetime', axis=1)
    test_pred = pipeline.predict(dftm)
    test_pred[test_pred < 0] = 0
    str_prediction = ""
    if ini:
        str_prediction = "datetime,count\n"
        ini = False
    for j in range(0, len(test_pred)):
        datetime = dates[j]
        pred = int(round(test_pred[j]))
        str_prediction += "{},{}\n".format(datetime, pred)
    f = open('pred_keras2.csv','a')
    f.write(str_prediction)
    f.close()

2011-01-31 23:59:59
2011-02-28 23:59:59
2011-03-31 23:59:59
2011-04-30 23:59:59
2011-05-31 23:59:59
2011-06-30 23:59:59
2011-07-31 23:59:59
2011-08-31 23:59:59
2011-09-30 23:59:59
2011-10-31 23:59:59
2011-11-30 23:59:59
2011-12-31 23:59:59
2012-01-31 23:59:59
2012-02-29 23:59:59
2012-03-31 23:59:59
2012-04-30 23:59:59
2012-05-31 23:59:59
2012-06-30 23:59:59
2012-07-31 23:59:59
2012-08-31 23:59:59
2012-09-30 23:59:59
2012-10-31 23:59:59
2012-11-30 23:59:59
2012-12-31 23:59:59


In [36]:
ranges = pd.date_range('2010-12-31 23:59:59', periods=25, freq='M').tolist()
dftm = dft[(dft['datetime'] > ranges[1]) & (dft['datetime'] < ranges[2])].copy()
dates = dftm['datetime'].tolist()
dates[0]

Timestamp('2011-02-20 00:00:00')